In [1]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import sys
import tensorflow as tf
import math
import numpy as np
import imageio
import json
import random
import time
from run_nerf_helpers import *
from load_llff import load_llff_data
from load_deepvoxels import load_dv_data
from load_blender import load_blender_data

In [2]:
tf.compat.v1.enable_eager_execution()

In [31]:
model_nerf = init_nerf_model(D = 8, W = 256, input_ch=63, input_ch_views=27, output_ch=5)
model_nerf.summary()

MODEL_NERF 63 27 <class 'int'> <class 'int'> False
(None, 90) (None, 63) (None, 27)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 90)]         0                                            
__________________________________________________________________________________________________
tf.split_3 (TFOpLambda)         [(None, 63), (None,  0           input_layer[0][0]                
__________________________________________________________________________________________________
layer_1 (Dense)                 (None, 256)          16384       tf.split_3[0][0]                 
__________________________________________________________________________________________________
layer_2 (Dense)                 (None, 256)          65792       layer_1[0][0]                    
________

In [45]:
def load_weight_from_old_trained_model(path):
    old_model_layers = ['layer_1/kernel:0', 'layer_1/bias:0', 'layer_2/kernel:0', 'layer_2/bias:0', 'layer_3/kernel:0',        
    'layer_3/bias:0', 'layer_4/kernel:0', 'layer_4/bias:0', 'layer_5/kernel:0', 'layer_5/bias:0', 'layer_6/kernel:0',        
    'layer_6/bias:0', 'layer_7/kernel:0', 'layer_7/bias:0', 'layer_8/kernel:0', 'layer_8/bias:0', 'bottleneck/kernel:0', 
    'bottleneck/bias:0', 'mu_layer_1/kernel:0', 'mu_layer_1/bias:0', 'pre_rgb_layer_1/kernel:0', 'pre_rgb_layer_1/bias:0',
    'mu_layer_2/kernel:0', 'mu_layer_2/bias:0', 'rgb_out/kernel:0', 'rgb_out/bias:0', 'density_out/kernel:0', 'density_out/bias:0',
    'mu_out/kernel:0', 'mu_out/bias:0']
    print(len(old_model_layers))

    weights = np.load(path, allow_pickle=True)
    print(len(weights)) 

    trainable_layers = [l.name for l in model_nerf.trainable_variables]
    valid_indices = []
    for i, l in enumerate(old_model_layers):
        if l in trainable_layers:
            valid_indices.append(i)

    weights = weights[valid_indices]
    print(len(weights))  

    return weights


In [46]:
trained_model_weights = 'trained/model_150000.npy'
weights = load_weight_from_old_trained_model(trained_model_weights)
model_nerf.set_weights(weights)

30
30
24


In [48]:
model_nerf.set_weights(weights)
save_loc = 'trained/transferred/model_150000.npy'
np.save(save_loc, model_nerf.get_weights())
model_nerf.set_weights(np.load(save_loc, allow_pickle=True))

In [43]:
model_nerf_fine = init_nerf_model(D = 8, W = 256, input_ch=63, input_ch_views=27, output_ch=5)
model_nerf_fine.summary()

MODEL_NERF 63 27 <class 'int'> <class 'int'> False
(None, 90) (None, 63) (None, 27)
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 90)]         0                                            
__________________________________________________________________________________________________
tf.split_4 (TFOpLambda)         [(None, 63), (None,  0           input_layer[0][0]                
__________________________________________________________________________________________________
layer_1 (Dense)                 (None, 256)          16384       tf.split_4[0][0]                 
__________________________________________________________________________________________________
layer_2 (Dense)                 (None, 256)          65792       layer_1[0][0]                    
________

In [50]:
trained_model_weights_fine = 'trained/model_fine_150000.npy'
weights_fine = load_weight_from_old_trained_model(trained_model_weights_fine)
model_nerf_fine.set_weights(weights_fine)

30
30
24


In [51]:
model_nerf_fine.set_weights(weights_fine)
save_loc = 'trained/transferred/model_fine_150000.npy'
np.save(save_loc, model_nerf_fine.get_weights())
model_nerf_fine.set_weights(np.load(save_loc, allow_pickle=True))

C:\Program Files\Python\Python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [54]:
opt = np.load('trained/optimizer_150000.npy', allow_pickle=True)
opt.shape

(109,)

In [4]:
model_mu = init_mu_model(D = 8, W = 256, input_ch=39, input_ch_views=39, output_ch=5)
model_mu.summary()

MODEL_MU 39 39 <class 'int'> <class 'int'> False
(None, 78) (None, 39) (None, 39)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 78)]         0                                            
__________________________________________________________________________________________________
tf.split_1 (TFOpLambda)         [(None, 39), (None,  0           input_layer[0][0]                
__________________________________________________________________________________________________
tf.concat_3 (TFOpLambda)        (None, 78)           0           tf.split_1[0][0]                 
                                                                 tf.split_1[0][1]                 
__________________________________________________________________________________________________
layer_1 (D